When i try to use the [centermask2](https://github.com/youngwanLEE/centermask2/blob/master/configs/centermask/centermask_V_39_eSE_FPN_ms_3x.yaml) to train my model, an error occur "AttributeError: 'BitMasks' object has no attribute 'polygons'". So there is a need of converting original RLE mask to polygons to run the code.

First we need to install detectron2 for subsequent use

In [ ]:
! python -m pip -q install 'git+https://github.com/facebookresearch/detectron2.git'

# RLE to Polygon
The basic idea is to first convert RLE into bitmask, and then obtain the corresponding polygons by looking for contours on the bitmask.

In [ ]:
def polygonFromMask(maskedArr): # https://github.com/hazirbas/coco-json-converter/blob/master/generate_coco_json.py

    contours, _ = cv2.findContours(maskedArr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    segmentation = []
    for contour in contours:
        # Valid polygons have >= 6 coordinates (3 points)
        if contour.size >= 6:
            segmentation.append(contour.flatten().tolist())
    RLEs = mask_util.frPyObjects(segmentation, maskedArr.shape[0], maskedArr.shape[1])
    RLE = mask_util.merge(RLEs)
    # RLE = mask.encode(np.asfortranarray(maskedArr))
    area = mask_util.area(RLE)
    [x, y, w, h] = cv2.boundingRect(maskedArr)

    return segmentation[0] #, [x, y, w, h], area

# Polygon to RLE
Using the opposite method, we can also convert polygons to RLE. Through this inverse process, we can observe whether there is information loss during the conversion process.

In [ ]:
from detectron2.structures import polygons_to_bitmask
def polygon_to_rle(polygon: list, shape=(520, 704)):
    '''
    polygon: a list of [x1, y1, x2, y2,....]
    shape: shape of bitmask
    Return: RLE type of mask
    '''
    mask = polygons_to_bitmask([np.asarray(polygon) + 0.25], shape[0], shape[1]) # add 0.25 can keep the pixels before and after the conversion unchanged
    rle = mask_util.encode(np.asfortranarray(mask))
    return rle

# Experiment

In [ ]:
import detectron2
from pathlib import Path
import random, cv2, os
import matplotlib.pyplot as plt
import numpy as np
import pycocotools.mask as mask_util
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer, default_setup, hooks, launch
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2.evaluation.evaluator import DatasetEvaluator
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling import GeneralizedRCNNWithTTA
setup_logger()

## Load Data

In [ ]:
dataDir=Path('/kaggle/input/sartorius-cell-instance-segmentation/')
train_json_file = "/kaggle/input/sartorius-cell-instance-segmentation-coco/annotations_train.json"
val_json_file = "/kaggle/input/sartorius-cell-instance-segmentation-coco/annotations_val.json"
register_coco_instances("sar_train", {}, train_json_file, dataDir)
register_coco_instances("sar_val", {}, val_json_file, dataDir)

In [ ]:
metadata = MetadataCatalog.get('sar_train')
train_ds = DatasetCatalog.get('sar_train')

In [ ]:
sample = train_ds[35]
sample_seg = sample["annotations"][0]["segmentation"]
sample_seg

In [ ]:
sample_mask = mask_util.decode(sample_seg)
sample_polygon = polygonFromMask(sample_mask)

In [ ]:
restored_rle = polygon_to_rle(sample_polygon)
restored_rle

# The information loss due to convertion
We will compare the mask before and after conversion in polygon format

In [ ]:
poly = polygonFromMask(mask_util.decode(restored_rle))
np.array(poly),np.array(sample_polygon)

I tested several different masks, and the results are roughly the same. The converted mask will lose a few pixels.
Noticed that in `polygon_to_rle`:
```python
mask = polygons_to_bitmask([np.asarray(polygon) + 0.25], shape[0], shape[1])
```
*0.25* is tried out through experimentation. When I set it to 0, I find that the x coordinate will differ by 1, and when it is set to 0.5, the y coordinate will differ by 1. So I took their average to alleviate this problem.

## Visualization

You can see that the restored RLE(**b'YW[52P`08N1N3N2M5L3N000002M<E\\Ud5'**) is different from the one(**b'YW[53o?8N1N3N2M5L3N000000O3N;DTec5'**) in the original data. We can visualize it and observe it more intuitively

In [ ]:
poly_mask = mask_util.decode(restored_rle)
_, ax = plt.subplots(1, 2, figsize=(40, 16))
ax[0].imshow(poly_mask, cmap="gray")
ax[1].imshow(sample_mask, cmap="gray")
ax[0].set_title("restored")
ax[1].set_title("ori")

Although there are differences in data, they seem to be similar. So information loss will not affect training.

## Save annotation in polygon format
I saved the converted result as a new file and placed it in the **cellis** dataset

In [ ]:
import json

def polygons_format_json(json_file, save_dir):
    with open(json_file) as f:
        imgs_anns = json.load(f)

    for idx, v in enumerate(imgs_anns["annotations"]):
        rle = v["segmentation"]
        compressed_rle = mask_util.frPyObjects(rle, rle.get('size')[0], rle.get('size')[1])
        mymask = mask_util.decode(compressed_rle)
        polygons = polygonFromMask(mymask)
        v["segmentation"] = [polygons]
    
    json_str = json.dumps(imgs_anns, indent=4, separators=(',', ':'))
    with open(save_dir, 'w') as json_file:
        json_file.write(json_str)

In [ ]:
os.mkdir("/kaggle/working/annotations")
for d in ["train", "val"]:
    json_file = f"/kaggle/input/sartorius-cell-instance-segmentation-coco/annotations_{d}.json"
    save_dir = f"/kaggle/working/annotations_{d}_polygon.json"
    polygons_format_json(json_file, save_dir)

## Display a sample file to check the convertion

In [ ]:
train_json_file = "../input/cellis/annotations/annotations_train_polygon.json"
val_json_file = "../input/cellis/annotations/annotations_val_polygon.json"
register_coco_instances("sar_train_polygon", {}, train_json_file, dataDir)
register_coco_instances("sar_val_polygon", {}, val_json_file, dataDir)

In [ ]:
def ann_visualization(data, metadata, index):
    data = data[index]
    img = cv2.imread(data["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata)
    out = visualizer.draw_dataset_dict(data)
    return out

In [ ]:
before = DatasetCatalog.get('sar_train')
after = DatasetCatalog.get('sar_train_polygon')
mb, ma = MetadataCatalog.get('sar_train'), MetadataCatalog.get('sar_train_polygon')

In [ ]:
index = 32
outb = ann_visualization(before, mb, index)
outa = ann_visualization(after, ma, index)
_, ax = plt.subplots(1, 2, figsize=(40, 30))
ax[0].imshow(outb.get_image()[:, :, ::-1])
ax[1].imshow(outa.get_image()[:, :, ::-1])

It seems that the convertion is successful!